# 差分方程 Difference Equation
《Python数学建模算法与应用》中介绍了差分方程的稳定性，线性差分方程的解法，有兴趣可以去看看，这里仅介绍一些有趣的例子

## 线性差分方程

### Logistic人口模型的差分实现
$$
\begin{cases} \dot{x} = r (1 - {x \over x_m}) x \\
              x(t_0) = x_0 \end{cases}
$$
不管这个方程组是如写出的，这可以按微分方程的方法去解，但如果以月/年为单位将时间离散，则可写出离散形式的方程。
$$
\begin{cases} a_{k+1} = (\Delta t \space r + 1) a_k - { r \over x_m} a_k^2 \\
              a_0 = x_0 \end{cases} \tag{1}
$$
这是向前差分，当然还有向后差分，这里不写。将$(1)$式写得更紧凑些，并给出数学问题：
$$
\begin{cases} a_{k+1} = \alpha a_k + \beta a_k^2 \\
              a_0 = x_0 \end{cases} \tag{2}
$$
已知 $ a_k, k = 0, 1, ...,5$。预测之后的 $ a_k $
其实就是找到 $ \alpha, \beta $ 使其更贴合给定数据，观察到未知量之间的线性关系，可用最小二乘法，这里为了更清楚地说明，特意写出矩阵形式：
$$
\begin{bmatrix}
    {a_0}&{a_0^2}\\
    {a_1}&{a_1^2}\\
    {\vdots}&{\vdots}\\
    {a_{5}}&{a_{5}^2}\\
\end{bmatrix}
\begin{bmatrix}
    {\alpha}\\
    {\beta}
\end{bmatrix} = 
\begin{bmatrix}
    {a_1}\\
    {a_2}\\
    {\vdots}\\
    {a_{6}}
\end{bmatrix} \tag{3}
$$
$(3)$ 式为超定线性方程组，接下来写程序

In [2]:
import numpy as np
from rich import print

a: list = [3.9, 5.3, 7.2, 9.6, 12.9, 17.1]
a: np.ndarray = np.array(a)
A = np.vstack([a[:-1], a[:-1] ** 2])  # 理解就行
print(A)

[[  3.9    5.3    7.2    9.6   12.9 ]
 [ 15.21  28.09  51.84  92.16 166.41]]

In [3]:
print(A := A.T)  # 转置

[[  3.9   15.21]
 [  5.3   28.09]
 [  7.2   51.84]
 [  9.6   92.16]
 [ 12.9  166.41]]

In [8]:
B = a.reshape(a.shape[0], 1)[:-1]  # 转置加切
print(B)

[[ 3.9]
 [ 5.3]
 [ 7.2]
 [ 9.6]
 [12.9]]

In [12]:
X = np.linalg.pinv(A) @ B
print(X)

[[1.00000000e+00]
 [6.89100153e-18]]

求出来了，不过未了加深对最小二乘拟合的印象，这里再“手算”一下
$$
A \cdot X = B
$$

A 列满秩，方程组超定
$$
X = (A^T A)^{-1} A^T B
$$

In [26]:
X_2 = np.linalg.inv(A.T.dot(A)).dot(A.T).dot(B)
print(X_2)

[[ 1.00000000e+00]
 [-4.60180645e-17]]

In [28]:
X_3 = ((A.T @ A) ** -1) @ A.T @ B  # @ 为 点乘
print(X_3)

[[2.        ]
 [0.18846558]]

In [27]:
X_4 = (A.T.dot(A) ** -1).dot(A.T).dot(B)
print(np.linalg.inv(np.dot(A.T, A)) @ A.T @ A)
print((A.T.dot(A) ** -1) @ A.T @ A )  # -1 是对每个元素求倒，要用inv
print(X_4)

[[ 1.00000000e+00 -2.28066797e-14]
 [-4.60180645e-17  1.00000000e+00]]

[[ 2.         21.25627642]
 [ 0.18846558  2.        ]]

[[2.        ]
 [0.18846558]]

### 斐波那契（Fibonacci）数列通项
$$
\begin{cases}
F_{k+2} = F_{k+1} + F_{k} \\
F_1 = 0 \\
F_0 = 1
\end{cases}
$$
写出差分方程形式：
$$
\begin{cases}
F_{k+1} = F_{k+1} \\
F_{k+2} = F_{k+1} + F_{k}
\end{cases}
$$

再写成矩阵形式！
$$
B_{k+1} = A \cdot B_{k}
$$
之后的不写了，求等比数列